In [3]:
import pandas as pd
from pathlib import Path

BASE = Path("../data/clean/vct_2024")
DIM = Path("../data/gold/dimensions")
OUT = Path("../data/gold/facts")

OUT.mkdir(parents=True, exist_ok=True)

In [4]:
# Silver source
agent_stats = pd.read_parquet(
    BASE / "agents/agents_pick_rates.parquet"
)

# Dimensions
dim_agent = pd.read_parquet(DIM / "dim_agent.parquet")
dim_map = pd.read_parquet(DIM / "dim_map.parquet")
dim_tournament = pd.read_parquet(DIM / "dim_tournament.parquet")

In [8]:
agent_stats = agent_stats[
    (agent_stats["map_name"] != "All Maps") &
    (agent_stats["stage_name"] != "All Stages") &
    (agent_stats["match_type_name"] != "All Match Types")
].copy()

In [10]:
fact = agent_stats.merge(
    dim_agent,
    on="agent_name",
    how="left"
)

In [11]:
print(fact["agent_id"].isna().sum())

0


In [12]:
fact = fact.merge(
    dim_map,
    on="map_name",
    how="left"
)

In [13]:
print(fact["map_id"].isna().sum())

0


In [14]:
fact = fact.merge(
    dim_tournament,
    on=[
        "tournament_name",
        "stage_name",
        "match_type_name"
    ],
    how="left"
)

In [16]:
print(fact["tournament_id"].isna().sum())

0


In [18]:
fact_agent_meta = fact[
    [
        # Keys
        "agent_id",
        "map_id",

        "tournament_id",
        "stage_id",
        "match_type_id",

        # Metric
        "pick_rate",
    ]
].copy()

In [20]:
out_path = OUT / "fact_agent_meta.parquet"

fact_agent_meta.to_parquet(
    out_path,
    index=False
)